In [1]:
import sqlite3
from flask import Flask
from flask import request,redirect
from flask import render_template,url_for
import subprocess as sp
import os
import pandas as pd

In [2]:
def connect_to_db(db,area,month1,month2,month3):
    with sqlite3.connect(db) as conn :
        cursor = conn.cursor()
        data = cursor.execute('SELECT temp from temp where area = "%s" and month in ("%s","%s","%s") ;'%(area,month1,month2,month3))
    avg = 0
    cnt = 1
    tmp = []
    for i in data :
        tmp.append(i[0])
        cnt+=1
        avg+=float(i[0])
    if cnt < 3 :
        return 'Please select 3 different month.'
    avg = avg/3
    return '%s : %s ,  %s : %s , %s : %s , average : %s' %(month1,tmp[0],month2,tmp[1],month3,tmp[2],avg)

In [3]:
## write avg_temp data to html
def temp_html(db,name):
    with sqlite3.connect(db) as conn :
        command = """SELECT * FROM temp"""
        data = pd.read_sql(command,conn)
    with open(os.path.join(os.getcwd(),'templates')+'/%s.html'%name,'w',encoding ='utf-8' ) as f :
        f.write(data.to_html())

In [4]:
temp_html('avg_temp.db','2')
temp_html('high_temp.db','3')
app = Flask(__name__)
## index
@app.route('/')
def index() :
    return render_template('1.html')
## after submit , return to success and print ans
@app.route('/success/<string:ans>')
def success(ans):
    return ans
##  when submit , get data from database , redirect to success
@app.route('/submit',methods=['POST'])
def submit():
    if request.method == 'POST' :
        area,which,month1,month2,month3 = request.form['area'],request.form['which'],\
        request.form['month1'],request.form['month2'],request.form['month3']     
        if which == 'avg' :
            ans = connect_to_db('avg_temp.db',area,month1,month2,month3)
        else :
            ans = connect_to_db('higt_temp.db',area,month1,month2,month3)
    return redirect(url_for('success',ans=ans))
## show database of avg_temp
@app.route('/database/avg_temp/')
def database_avg_temp():
    return render_template('2.html')
## show database of higt_temp
@app.route('/database/high_temp/')
def database_high_temp():
    return render_template('3.html')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Feb/2022 15:50:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 15:50:20] "POST /submit HTTP/1.1" 302 -
127.0.0.1 - - [11/Feb/2022 15:50:20] "GET /success/四月%20:%2015.7%20%2C%20%20二月%20:%2021.4%20%2C%20七月%20:%2029.0%20%2C%20average%20:%2022.03333333333333 HTTP/1.1" 200 -
